# Capstone Project: Copenhagen Housing Prices

This is the project code to analyse houing prices in Copenhagen, Denmark.
It will use the city distric housing prices per square metre to find an affordable area for a customer.
It will also regard the price changes over the last three years to investigate the investment prospective in each city district.
As one of the most bike-friendly cities in the world it will investigate the venues 2km around each Metro station to make it easy for customers to find an area in each district suitable for them. 

In [1]:
import numpy as np
import pandas as pd


## Copenhagen city districts

download/import folium for the map

In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')


Folium installed and imported!


get the geojson for the copenhagen district boarders

In [3]:
#!pip install wget
import wget
print('wget installed and imported!')

wget installed and imported!


In [4]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [5]:
import urllib.request, json 
with urllib.request.urlopen("http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:bydel&outputFormat=json&SRSNAME=EPSG:4326") as url:
    ByDel = json.loads(url.read().decode())
    

Names of the City districts are (json names without danish letters)
1. Indre By 
2. Østerbro (sterbro)
3. Bispebjerg
4. Valby
5. Vesterbro-Kongens Enghave
6. Amager st
7. Brønshøj -Husum (Brnshj-Husum) 
8. Vanløse (Vanlse) 
9. Nørrebro (Nrrebro) 
10. Amager Vest

## Copenhagen housing prices 

In [6]:
# load the csv with housing data
filename='https://raw.githubusercontent.com/Ges-Wan/Coursera_Capstone/master/boligpris.csv'
HouseDat= pd.read_csv(filename)
# get the house prises from file
HousePrise=HouseDat.iloc[1::5,:]
#reindex the house price
HousePrise.reset_index(drop=True,inplace=True)



#add the quarter of the pricing
Date=HouseDat.iloc[::5,0].astype(str).values.tolist()
HousePrise.insert(0, "Price Date",Date, True)
HousePrise.head()

# rename the city quatters to be associatable with the danish geojson file names
# returns a warning from the danish special letters
HousePrise.rename(columns={'KbhK':'Indre By', 'Kbh V':'Vesterbro-Kongens Enghave','Kbh Ø':'sterbro','Kbh N':'Nrrebro','Kbh S':'Amager st','Kbh NV':'Bispebjerg','Kbh SV':'Amager Vest','Vanløse':'Vanlse','Brønshøj':'Brnshj-Husum'}, inplace=True)
HousePrise.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Price Date,Indre By,Vesterbro-Kongens Enghave,sterbro,Nrrebro,Amager st,Bispebjerg,Amager Vest,Valby,Vanlse,Brnshj-Husum
0,1. kv 2020,50.808,48.627,47.419,43.460,38.109,34.282,41.861,35.610,33.516,29.172
1,4. kv 2019,51.669,46.798,45.039,43.164,38.261,33.407,39.303,34.916,34.278,29.817
2,3. kv 2019,50.605,44.389,44.141,42.698,38.414,33.878,38.566,34.534,34.013,30.023
3,2. kv 2019,50.656,44.612,44.904,42.359,38.109,33.676,38.760,34.708,33.151,29.319
4,1. kv 2019,49.354,45.770,45.241,43.162,37.705,32.547,38.045,33.498,33.767,31.241
5,4. kv 2018,49.387,44.945,45.286,41.225,37.524,32.841,39.667,34.040,33.531,30.349
6,3. kv 2018,50.737,46.880,45.346,42.685,38.278,34.750,37.791,34.593,33.651,29.160
7,2. kv 2018,51.389,46.459,46.053,43.367,38.613,34.725,38.137,33.386,34.421,29.173
8,1. kv 2018,51.883,46.551,45.407,43.003,38.194,34.318,38.725,34.932,33.721,29.762
9,4. kv 2017,50.249,45.216,43.900,42.997,36.093,33.507,37.050,33.525,32.547,28.183


In [7]:
#creating two new data frames containing per city part:

# 1. the current price (1.kv 2020)
df_Price=pd.DataFrame(HousePrise.iloc[0,1:].transpose())
df_Price.reset_index(drop=False,inplace=True)
df_Price.columns = ['District','Price per sqm']
df_Price['Price per sqm'] = df_Price['Price per sqm'].astype(float)
df_Price.head()

# 2. the price change from average price of the last three years 2017 to 2019
Pr2015=HousePrise.iloc[17:21,1:].astype(float).mean()
Pr2019=HousePrise.iloc[1:5,1:].astype(float).mean()
df_PrIncr=pd.DataFrame((Pr2019-Pr2015)/Pr2015/5*100)
df_PrIncr.reset_index(drop=False,inplace=True)
df_PrIncr.columns = ['District','annual price increase in %']
df_PrIncr.head()

,District,annual price increase in %
0,Indre By,4.311520
1,Vesterbro-Kongens Enghave,4.442709
2,sterbro,4.563891
3,Nrrebro,5.784093
4,Amager st,6.219781


creating the two coropeth maps showing housing prices and price increase per year

In [8]:
# create a plain world map
longitude=12.568337
latitude=55.676098
CphPrice_map = folium.Map(location=[latitude, longitude], zoom_start=12)
# generate choropleth map using the housing prices of copenhagen
CphPrice_map.choropleth(
    geo_data=ByDel,
    data=df_Price,
    columns=['District', 'Price per sqm'],
    key_on='feature.properties.navn',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Price per square metre in 1000 DKK'
)

# display map
CphPrice_map

In [9]:
CphPriceInc_map = folium.Map(location=[latitude, longitude], zoom_start=12)
# generate choropleth map using the housing price increase for copenhagen
CphPriceInc_map.choropleth(
    geo_data=ByDel,
    data=df_PrIncr,
    columns=['District', 'annual price increase in %'],
    key_on='feature.properties.navn',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='annual price increase in %'
)

# display map
CphPriceInc_map

## Venues in each city district at 2km distance from the metro stations 


Getting the list of metro stations

In [10]:
# Scrape the names and hyperlinks with BeautifulSoup

# import the library we use to open URLs
import urllib.request
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
# to extract substrings
import re

# 1. specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_Copenhagen_Metro_stations"
# 2. open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# 3. parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "html.parser")
# 4. use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
# 5. use the 'find function' to bring back the wiki sort table
right_table=soup.find('table', class_='wikitable plainrowheaders sortable')

# 6. identify the row entries of each collumn
Sta=[]
hyref=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('th')
    m = re.search('title="(.+?)">', str(cells))
    if m:
        found = m.group(1)
        Sta.append(found)
    n = re.search('href="(.+?)" ', str(cells))
    if n:
        found = n.group(1)
        hyref.append('https://en.wikipedia.org'+found)

        
# 7. build the dataframe from the scaped data
dfMetro=pd.DataFrame(Sta,columns=['Station'])
dfMetro['Link']=hyref


# 8. delete the orient station as the we can not retrieve data for it
dfMetro.drop(dfMetro[dfMetro.Station=='Orientkaj Station (page does not exist)'].index, axis=0, inplace=True)
dfMetro.reset_index(drop=True,inplace=True)
dfMetro.head()

,Station,Link
0,Aksel Møllers Have Station,https://en.wikipedia.org/wiki/Aksel_M%C3%B8lle...
1,Amager Strand Station,https://en.wikipedia.org/wiki/Amager_Strand_St...
2,Amagerbro Station,https://en.wikipedia.org/wiki/Amagerbro_Station
3,Bella Center Station,https://en.wikipedia.org/wiki/Bella_Center_Sta...
4,Christianshavn Station,https://en.wikipedia.org/wiki/Christianshavn_S...


Getting the longitude and latitude for each metro station

In [11]:
# Scrape the coordinates from the hyperlinks with BeautifulSoup
latitudeMet=[]
longitudeMet=[]

for index, row in dfMetro.iterrows():

    # 1. specify which URL/web page we are going to be scraping
    url = dfMetro.iloc[index,1]
    # 2. open the url using urllib.request and put the HTML into the page variable
    page = urllib.request.urlopen(url)
    # 3. parse the HTML from our URL into the BeautifulSoup parse tree format
    soup = BeautifulSoup(page, "html.parser")
    # 4. use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
    all_tables=soup.find_all("table")
    # 5. use the 'find function' to bring back the info table
    right_table=soup.find('table', class_='infobox vcard')
    # 6. extract location data for each station
    m = re.search('<span class="geo-dec" title="Maps, aerial photos, and other data for this location">(.+?)°N ', str(right_table))
    if m:
        found = m.group(1)
        latitudeMet.append(found)
    else: 
        latitudeMet.append('not found')

    n = re.search('°N (.+?)°E', str(right_table))
    if n:
        found = n.group(1)
        longitudeMet.append(found)
    else: 
        longitudeMet.append('not found')
# add the columns to the dataset
dfMetro['Longitude']=longitudeMet
dfMetro['Latitude']=latitudeMet

# drop the stations under construction where gps data is not yet available
dfMetro.drop(dfMetro[dfMetro.Longitude=='not found'].index, axis=0, inplace=True)
dfMetro.reset_index(drop=True,inplace=True)

# drop the link as it is no longer needed
dfMetro.drop(columns=['Link'],inplace=True)

# convert geo data to nummeric data
dfMetro[['Longitude', 'Latitude']] = dfMetro[['Longitude', 'Latitude']].apply(pd.to_numeric)

dfMetro.head()

,Station,Longitude,Latitude
0,Aksel Møllers Have Station,12.533361,55.686444
1,Amager Strand Station,12.631670,55.656110
2,Amagerbro Station,12.602944,55.663361
3,Bella Center Station,12.582944,55.638060
4,Christianshavn Station,12.591222,55.672220


## Getting the top 10 venues within biking distance for each Metro Station

getting the foursquare client (hidden cell)

In [12]:
# The code was removed by Watson Studio for sharing.

borrow the get Near by venues function 

In [13]:
import requests # library to handle requests

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

get the closest 100 venues for each station

In [15]:
LIMIT=100
Copenhagen_venues = getNearbyVenues(names=dfMetro['Station'],
                                  latitudes=dfMetro['Latitude'],
                                  longitudes=dfMetro['Longitude']
                                 )

Aksel Møllers Have Station
Amager Strand Station
Amagerbro Station
Bella Center Station
Christianshavn Station
DR Byen Station
Enghave Plads Station
Fasanvej Station
Femøren Station
Flintholm Station
Forum Station
Frederiksberg Station
Frederiksberg Allé Station
Islands Brygge Station
Kastrup Station
Copenhagen Central Station
Kongens Nytorv Station
Lergravsparken Station
Lindevang Station
Lufthavnen Station
Nordhavn Station
Nørrebro Station
Nørrebros Runddel Station
Nørreport Station
Nuuks Plads Station
Øresund Station
Østerport Station
Poul Henningsens Plads Station
Skjolds Plads Station
Sundby Station
Trianglen Station
Vanløse Station
Vestamager Station
Vibenshus Runddel Station


In [16]:
#check the resultign data frame
print(Copenhagen_venues.shape)
Copenhagen_venues.head()

(3289, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aksel Møllers Have Station,55.686444,12.533361,The Coffee Collective,55.686960,12.533222,Coffee Shop
1,Aksel Møllers Have Station,55.686444,12.533361,Forno a Legna,55.682382,12.535324,Pizza Place
2,Aksel Møllers Have Station,55.686444,12.533361,Brødflov,55.681983,12.534823,Bakery
3,Aksel Møllers Have Station,55.686444,12.533361,Gensyn Bar,55.684205,12.543145,Cocktail Bar
4,Aksel Møllers Have Station,55.686444,12.533361,Frederiksberg Hovedbibliotek,55.680724,12.530827,Library


get the number of venues per station and add the number to the Metro data frame

In [17]:
CphVen=pd.DataFrame(Copenhagen_venues.groupby('Neighborhood').count().Venue)
CphVen.head()


,Venue
Neighborhood,
Aksel Møllers Have Station,100
Amager Strand Station,100
Amagerbro Station,100
Bella Center Station,74
Christianshavn Station,100


analyse each neighbrourhood to see the cathegory of venues around each station

In [18]:
# one hot encoding and grouping of venues
Cph_onehot = pd.get_dummies(Copenhagen_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Cph_onehot['Neighborhood'] = Copenhagen_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Cph_onehot.columns[-1]] + list(Cph_onehot.columns[:-1])
Cph_onehot = Cph_onehot[fixed_columns]


Cph_grouped = Cph_onehot.groupby('Neighborhood').mean().reset_index()
Cph_grouped.head()

,Neighborhood,Zoo Exhibit,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Aksel Møllers Have Station,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.00,0.01,0.00,0.03,0.01,0.00,0.00,0.01
1,Amager Strand Station,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.010000,0.01,...,0.0,0.0,0.00,0.00,0.01,0.01,0.00,0.00,0.01,0.00
2,Amagerbro Station,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.01,0.00,0.00,0.04,0.02,0.00,0.00,0.00
3,Bella Center Station,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.013514,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Christianshavn Station,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.01,0.00,0.00,0.04,0.02,0.01,0.00,0.00


create function to get the top 10 venues in sorted order

In [19]:
# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the neighborhood data frame

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Cph_grouped['Neighborhood']

for ind in np.arange(Cph_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Cph_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aksel Møllers Have Station,Café,Coffee Shop,Beer Bar,Bakery,Park,Cocktail Bar,Scandinavian Restaurant,Pub,Wine Bar,Italian Restaurant
1,Amager Strand Station,Beach,Bakery,Coffee Shop,Café,Pizza Place,Gym / Fitness Center,Sushi Restaurant,Burger Joint,Grocery Store,Supermarket
2,Amagerbro Station,Bakery,Coffee Shop,Café,Scandinavian Restaurant,Pizza Place,Bar,Concert Hall,Wine Bar,Gym / Fitness Center,Sushi Restaurant
3,Bella Center Station,Restaurant,Supermarket,Other Great Outdoors,Park,Sporting Goods Shop,Convenience Store,Hotel,Café,Discount Store,Pizza Place
4,Christianshavn Station,Coffee Shop,Scandinavian Restaurant,Bar,Café,Hotel,Wine Bar,Bakery,Ice Cream Shop,Theater,Beer Bar


create a map with the top venues as pop-up down for the markers
sort the marker color by grouped neighborhoods to make it easy to find your favorit

In [21]:
# set number of clusters
kclusters = 5

Cph_grouped_clustering = Cph_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cph_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 4, 0, 0, 1, 2, 2, 3], dtype=int32)

join the data frames for plotting on top of the map

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Cph_merged = dfMetro
Cph_merged.rename(columns={"Station": "Neighborhood"},inplace=True)

# merge cph_grouped with cph_data to add latitude/longitude for each neighborhood
Cph_merged = Cph_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Cph_merged.head() 

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aksel Møllers Have Station,12.533361,55.686444,2,Café,Coffee Shop,Beer Bar,Bakery,Park,Cocktail Bar,Scandinavian Restaurant,Pub,Wine Bar,Italian Restaurant
1,Amager Strand Station,12.631670,55.656110,1,Beach,Bakery,Coffee Shop,Café,Pizza Place,Gym / Fitness Center,Sushi Restaurant,Burger Joint,Grocery Store,Supermarket
2,Amagerbro Station,12.602944,55.663361,1,Bakery,Coffee Shop,Café,Scandinavian Restaurant,Pizza Place,Bar,Concert Hall,Wine Bar,Gym / Fitness Center,Sushi Restaurant
3,Bella Center Station,12.582944,55.638060,4,Restaurant,Supermarket,Other Great Outdoors,Park,Sporting Goods Shop,Convenience Store,Hotel,Café,Discount Store,Pizza Place
4,Christianshavn Station,12.591222,55.672220,0,Coffee Shop,Scandinavian Restaurant,Bar,Café,Hotel,Wine Bar,Bakery,Ice Cream Shop,Theater,Beer Bar


create the map that shows the prices as well as the marker with right cluster color
when you click on the marker the top 5 venues should appear

In [24]:
# copenhagen price map


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
markers_colors = []
for lat, lng, station, cluster,V01,V02,V03,V04,V05 in zip(Cph_merged['Latitude'], Cph_merged['Longitude'], Cph_merged['Neighborhood'],Cph_merged['Cluster Labels'],
                                                          Cph_merged['1st Most Common Venue'], Cph_merged['2nd Most Common Venue'],Cph_merged['3rd Most Common Venue'],Cph_merged['4th Most Common Venue'],Cph_merged['5th Most Common Venue']):
    label = '{}:\n1. {}\n2.{}\n3.{}\n4.{}\n5.{}'.format(station,V01,V02,V03,V04,V05)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.7,
        parse_html=False).add_to(CphPrice_map)  
    
CphPrice_map



    


